# Adaptive Fireplace Learning System - Live Demonstration

This notebook demonstrates the intelligent fireplace learning system that adapts to your specific fireplace's thermal characteristics through real-world usage patterns.

## Key Features
- 🔥 **Temperature Differential Learning**: Learns from actual thermal measurements
- 🎯 **Real-Time Adaptation**: Improves accuracy with every fireplace session
- 🌡️ **Weather Correlation**: Discovers how outdoor temperature affects effectiveness
- ⚡ **No Historical Data Required**: Starts learning immediately
- 🛡️ **Safety Built-In**: Conservative estimates with safety bounds

In [ ]:
# Setup and imports
import sys
import os
sys.path.append('/opt/ml_heating')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import tempfile

from src.adaptive_fireplace_learning import (
    AdaptiveFireplaceLearning, 
    FireplaceObservation,
    integrate_adaptive_fireplace_with_multi_source_physics
)
from src.multi_heat_source_physics import MultiHeatSourcePhysics

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("🔥 Adaptive Fireplace Learning System Demo")
print("=" * 50)

## 1. Initialize Adaptive Learning System

First, let's create the adaptive fireplace learning system with your configuration:

In [ ]:
# Create temporary state file for demonstration
temp_state_file = tempfile.NamedTemporaryFile(delete=False, suffix='_fireplace_demo.json')
temp_state_file.close()

# Initialize adaptive fireplace learning
adaptive_fireplace = AdaptiveFireplaceLearning(state_file=temp_state_file.name)

print(f"📁 Demo state file: {temp_state_file.name}")
print(f"🔧 Fireplace ON threshold: {adaptive_fireplace.fireplace_on_threshold}°C differential")
print(f"🔧 Fireplace OFF threshold: {adaptive_fireplace.fireplace_off_threshold}°C differential")
print(f"📚 Learning rate: {adaptive_fireplace.learning_rate}")
print(f"🎯 Min observations for learning: {adaptive_fireplace.min_observations_for_learning}")

# Show initial learned coefficients
initial_coeffs = adaptive_fireplace.learning_state.learned_coefficients
print("\n🏠 Initial Conservative Estimates:")
for key, value in initial_coeffs.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")

## 2. Simulate Your Fireplace Control Logic

Let's simulate your sophisticated temperature differential detection:

In [ ]:
def simulate_fireplace_detection(living_room_temp, other_rooms_temp):
    """
    Simulate your fireplace detection logic:
    - ON:  living_room_temp - other_rooms_mean > 2.0°C
    - OFF: living_room_temp - other_rooms_mean < 0.8°C (hysteresis)
    """
    temp_differential = living_room_temp - other_rooms_temp
    
    if temp_differential > 2.0:
        return True, temp_differential, f"🔥 Fireplace detected ON (>{temp_differential:.1f}°C diff)"
    elif temp_differential < 0.8:
        return False, temp_differential, f"❄️ Fireplace detected OFF (<{temp_differential:.1f}°C diff)"
    else:
        return None, temp_differential, f"🔄 Hysteresis zone ({temp_differential:.1f}°C diff)"

# Test fireplace detection with example scenarios
test_scenarios = [
    {"living_room": 22.8, "other_rooms": 20.2, "description": "Fireplace starting up"},
    {"living_room": 24.5, "other_rooms": 20.5, "description": "Fireplace fully active"},
    {"living_room": 21.5, "other_rooms": 20.8, "description": "Fireplace cooling down"},
    {"living_room": 20.3, "other_rooms": 20.1, "description": "Fireplace off"},
]

print("🏠 Simulating Your Fireplace Detection Logic:")
print("-" * 60)
for scenario in test_scenarios:
    active, diff, message = simulate_fireplace_detection(
        scenario["living_room"], scenario["other_rooms"]
    )
    print(f"{scenario['description']:.<30} {message}")

## 3. Learning Progress Demo

Let's demonstrate the learning system:

In [ ]:
# Simulate a fireplace session
print("🔥 Adaptive Fireplace Learning Demo")
print("=" * 50)

# Test scenarios
scenarios = [
    {"living_room": 20.5, "other_rooms": 20.0, "outdoor": 5.0, "active": False, "desc": "Before fireplace"},
    {"living_room": 23.2, "other_rooms": 20.1, "outdoor": 5.0, "active": True, "desc": "Fireplace started"},
    {"living_room": 24.8, "other_rooms": 20.3, "outdoor": 5.0, "active": True, "desc": "Peak heating"},
    {"living_room": 21.5, "other_rooms": 20.8, "outdoor": 5.0, "active": True, "desc": "Cooling down"},
    {"living_room": 20.6, "other_rooms": 20.4, "outdoor": 5.0, "active": False, "desc": "Fireplace ended"}
]

for scenario in scenarios:
    result = adaptive_fireplace.observe_fireplace_state(
        living_room_temp=scenario['living_room'],
        other_rooms_temp=scenario['other_rooms'],
        outdoor_temp=scenario['outdoor'],
        fireplace_active=scenario['active']
    )
    
    print(f"\n{scenario['desc']}:")
    print(f"  Temperature differential: {result['temp_differential']:.1f}°C")
    print(f"  Heat contribution: {result['heat_contribution_kw']:.2f}kW")
    print(f"  Learning confidence: {result['learning_confidence']:.3f}")
    print(f"  Session status: {result['session_update']['status']}")

# Show learning summary
summary = adaptive_fireplace.get_learning_summary()
print(f"\n📊 Learning Summary:")
print(f"  Total observations: {summary['learning_status']['total_observations']}")
print(f"  Learning confidence: {summary['learning_status']['learning_confidence']:.3f}")
print(f"  Learning active: {summary['learning_status']['learning_active']}")

print("\n✅ Demo complete! The system learns from real fireplace usage.")